In [1]:
import os
import pickle
from copy import copy

import tensorflow as tf
import keras
import numpy as np

sess = tf.Session()
keras.backend.set_session(sess)
tf.keras.backend.set_session(sess)
tf.keras.backend._LEARNING_PHASE = tf.constant(0)
tf.keras.backend.set_learning_phase(0)

/Users/cole/.virtualenvs/keras-to-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/cole/.virtualenvs/keras-to-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/cole/.virtualenvs/keras-to-tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/cole/.virtualenvs/keras-to-tf/lib

In [2]:
orig_model = keras.models.load_model('model_1.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/Users/cole/.virtualenvs/keras-to-tf/lib/python3.7/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="data/glove.6B.50d.txt", word2vec_output_file="data/gensim_glove_vectors.txt")
from gensim.models.keyedvectors import KeyedVectors
wv = KeyedVectors.load_word2vec_format("data/gensim_glove_vectors.txt", binary=False)

In [4]:
test = []
for i in range(100):
    test.append(wv['python'])
test = np.array(test).reshape(1, 100, 50)

In [5]:
orig_model.predict(test)

array([[0.39491233]], dtype=float32)

In [6]:
# Maybe not necessary, but let's force everything into tf.keras

def convert_to_tf(model, name):    
    model.name = '{}'.format(str(name).lower().replace('(', '')
                                              .replace(')', '')
                                              .replace(',', '')
                                              .replace('/', '_')
                                              .replace(' ', '_'))
    js = model.to_json()
    filepath = '/tmp/{}_weights.h5'.format(model.name)
    model.save_weights(filepath)
    new_model = tf.keras.models.model_from_json(js)
    new_model.load_weights(filepath)

    models_dir = 'models'
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    new_model.save('{}/{}.h5'.format(models_dir, model.name))
    return new_model

In [7]:
tf_orig_model = convert_to_tf(orig_model, 'cnn_1')

In [8]:
def create_wv_embedding_layer(wv, oov_vector_file, pad_string='--PAD--', max_sequence_length=100):
    """
    From an existing gensim word2vec model create a tf.keras embedding layer to use ahead of pretrained models in tf serving
    Pad string will assigned a vector of zeros
    Out of Vocabulary vector would've been predefined when original model was created.  We need to keep it around.


    When saving a combined model:
        signature = tf.saved_model.signature_def_utils.predict_signature_def(
            inputs={'image': model.input}, outputs={'scores': model.output}
            )
        builder = tf.saved_model.builder.SavedModelBuilder('/tmp/wvmodel7')
        builder.add_meta_graph_and_variables(
            sess=K.get_session(),
            tags=[tf.saved_model.tag_constants.SERVING],
            signature_def_map={tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature},
            legacy_init_op=tf.tables_initializer()
            )
        builder.save()

    Args:
        wv                                  gensim word vector model
        oov_token_file                      str
        pad_string                          str
        max_sequence_lenght                 str

    Returns:
        tf.keras model that will take tokens as strings and return a matrix of word vectors
    """

    # Get the oov token
    with open(oov_vector_file, 'rb') as f:
        oov_vec = pickle.load(f)


    # Construct new index to word mapping
    index2word = copy(wv.index2word)
    index2word.append(pad_string)

    # Build the embedding matrix
    embedding_dim = wv[wv.index2word[0]].shape[0]
    embedding_matrix = np.zeros((len(wv.index2word) + 2, embedding_dim))  # +2 because 1 for pad token, 1 for oov
    for i, word in enumerate(wv.index2word):  # use the original lookup index as it doesn't have a pad token
        embedding_vector = wv[word]
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    # Set the last entry to the oov_token
    embedding_matrix[len(embedding_matrix) - 1] = oov_vec

    # Build the tf lookup table
    lookup_op = tf.contrib.lookup.index_table_from_tensor(tf.constant(index2word),  # not wv.index2word as we want the pad token
                                                          default_value=len(embedding_matrix) - 1,
                                                          name='lookup_op')  # points to the oov/unk token


    # Build the tf.keras Embedding layer
    embedding_layer = tf.keras.layers.Embedding(len(wv.index2word) + 2,
                                             embedding_dim,
                                             weights=[embedding_matrix],
                                             input_length=max_sequence_length,
                                             trainable=False,
                                             name='embedding_layer')

    # Build a model that takes tokens
    _input = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.string, name='wv_in')
    x = tf.keras.layers.Lambda(lookup_op.lookup, output_shape=(max_sequence_length,), name='lambda_lookup')(_input)
    x = embedding_layer(x)
    wv_model = tf.keras.models.Model(_input, x)  # Everything in tf.keras please

    lookup_op.init.run(session=tf.keras.backend.get_session())

#     tf.compat.v1.tables_initializer().run(session=tf.keras.backend.get_session())

    

    return wv_model

In [9]:
OOV_VECTOR_FILE = 'data/oov_vector.p'
embedding = create_wv_embedding_layer(wv, OOV_VECTOR_FILE, max_sequence_length=100)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `initializer` instead.


In [10]:
embedding.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wv_in (InputLayer)           (None, 100)               0         
_________________________________________________________________
lambda_lookup (Lambda)       (None, 100)               0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 100, 50)           20000100  
Total params: 20,000,100
Trainable params: 0
Non-trainable params: 20,000,100
_________________________________________________________________


In [11]:
# with tf.Session() as sess:
#     legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
#     init_op = tf.group(tf.local_variables_initializer())
#     sess.run(init_op)
vec = embedding.predict(np.array([['python'] * 100]))

In [12]:
vec

array([[[ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025],
        [ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025],
        [ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025],
        ...,
        [ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025],
        [ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025],
        [ 0.5897 , -0.55043, -1.0106 , ...,  0.15425, -0.93256,
         -0.15025]]], dtype=float32)

In [13]:
wv['python']

array([ 0.5897  , -0.55043 , -1.0106  ,  0.41226 ,  0.57348 ,  0.23464 ,
       -0.35773 , -1.78    ,  0.10745 ,  0.74913 ,  0.45013 ,  1.0351  ,
        0.48348 ,  0.47954 ,  0.51908 , -0.15053 ,  0.32474 ,  1.0789  ,
       -0.90894 ,  0.42943 , -0.56388 ,  0.69961 ,  0.13501 ,  0.16557 ,
       -0.063592,  0.35435 ,  0.42819 ,  0.1536  , -0.47018 , -1.0935  ,
        1.361   , -0.80821 , -0.674   ,  1.2606  ,  0.29554 ,  1.0835  ,
        0.2444  , -1.1877  , -0.60203 , -0.068315,  0.66256 ,  0.45336 ,
       -1.0178  ,  0.68267 , -0.20788 , -0.73393 ,  1.2597  ,  0.15425 ,
       -0.93256 , -0.15025 ], dtype=float32)

In [14]:
MAX_TOKENS = 100

_input = tf.keras.layers.Input(shape=(MAX_TOKENS,), dtype=tf.string)
x = embedding(_input)
x = tf_orig_model(x)

model = tf.keras.models.Model(_input, x)

In [15]:
legacy_init_op = tf.group([tf.tables_initializer(), tf.local_variables_initializer()], name='legacy_init_op')
init_op = tf.group(tf.local_variables_initializer())
init_op.run(session=tf.keras.backend.get_session())

In [16]:
!rm -rf models/blog_model

In [17]:
tf.keras.backend.set_learning_phase(0)
outputs = {'blog_model': model.outputs[0]}
signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={'tokens': model.input}, outputs=outputs
    )
builder = tf.saved_model.builder.SavedModelBuilder('models/blog_model')
builder.add_meta_graph_and_variables(
    sess=tf.keras.backend.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature},
    legacy_init_op=legacy_init_op
    )
builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/blog_model/saved_model.pb


b'models/blog_model/saved_model.pb'

In [18]:
text = 'python ' * 100
tokens = text.split()
np.array([tokens]).shape

(1, 100)

In [19]:
model.predict(np.array(tokens).reshape((1, -1)))

array([[0.39491233]], dtype=float32)

In [20]:
wv['python']

array([ 0.5897  , -0.55043 , -1.0106  ,  0.41226 ,  0.57348 ,  0.23464 ,
       -0.35773 , -1.78    ,  0.10745 ,  0.74913 ,  0.45013 ,  1.0351  ,
        0.48348 ,  0.47954 ,  0.51908 , -0.15053 ,  0.32474 ,  1.0789  ,
       -0.90894 ,  0.42943 , -0.56388 ,  0.69961 ,  0.13501 ,  0.16557 ,
       -0.063592,  0.35435 ,  0.42819 ,  0.1536  , -0.47018 , -1.0935  ,
        1.361   , -0.80821 , -0.674   ,  1.2606  ,  0.29554 ,  1.0835  ,
        0.2444  , -1.1877  , -0.60203 , -0.068315,  0.66256 ,  0.45336 ,
       -1.0178  ,  0.68267 , -0.20788 , -0.73393 ,  1.2597  ,  0.15425 ,
       -0.93256 , -0.15025 ], dtype=float32)